In [14]:
import numpy as np
import torch
import theseus as th
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from scipy.spatial.transform import Rotation
from typing import Union, List, Tuple, Optional, cast

In [15]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def torch2np(tensor: torch.Tensor) -> np.ndarray:
    """ Converts a PyTorch tensor to a NumPy ndarray.
    Args:
        tensor: The PyTorch tensor to convert.
    Returns:
        A NumPy ndarray with the same data and dtype as the input tensor.
    """
    return tensor.detach().cpu().numpy()

In [21]:
class GaussianSLAMEdge:
    def __init__(
        self,
        vertex_idx_i: int,
        vertex_idx_j: int,
        relative_pose: th.SE3,
        cost_weight: th.CostWeight
    ):
        self.vertex_idx_i = vertex_idx_i
        self.vertex_idx_j = vertex_idx_j
        self.relative_pose = relative_pose
        self.cost_weight = cost_weight

    def to(self, *args, **kwargs):
        self.weight.to(*args, **kwargs)
        self.relative_pose.to(*args, **kwargs)


class GaussianSLAMPoseGraph:
    def __init__(
        self, 
        requires_auto_grad = True
    ):
        self._requires_auto_grad = requires_auto_grad
        self._objective = th.Objective()
        self._theseus_inputs = {} 

    def add_odometry_edge(
            self,
            vertex_i: th.SE3,
            vertex_j: th.SE3,
            edge: GaussianSLAMEdge,
            gaussian_means: torch.Tensor
        ):

        relative_pose, cost_weight = edge.relative_pose, edge.cost_weight

        #gaussian_means_th = th.Point3(
            #tensor=gaussian_means,
            #name=f"gaussian_means_odometry__{edge.vertex_idx_i}_{edge.vertex_idx_j}")

        optim_vars = vertex_i, vertex_j
        #aux_vars = relative_pose, gaussian_means_th
        if self._requires_auto_grad:
            for point in gaussian_means:
                point_th = th.Point3(tensor=point)
                aux_vars = relative_pose, point_th
                cost_function = th.AutoDiffCostFunction( # is the 3rd input correct?
                    optim_vars, dense_surface_alignment, 3, cost_weight, aux_vars
                )
                self._objective.add(cost_function)
                
            self._theseus_inputs.update({
                vertex_i.name: vertex_i.tensor, 
                vertex_j.name: vertex_j.tensor
            })
        else:
            raise NotImplementedError()

    def add_loop_closure_edge(
            self,
            vertex_i: th.SE3,
            vertex_j: th.SE3,
            edge: GaussianSLAMEdge,
            gaussian_means: torch.tensor,
            coefficient: float # hyperparameter, not the same as in the paper
        ):

        relative_pose = edge.relative_pose
        cost_weight_alignment = edge.cost_weight # for dense surface alignment

        l_ij = th.Vector(tensor=torch.ones(1, 1), name=f"line_process_{edge.vertex_idx_i}_{edge.vertex_idx_j}")
        #gaussian_means_th = th.Point3(
            #tensor=gaussian_means, 
            #name=f"gaussian_means_odometry__{edge.vertex_idx_i}_{edge.vertex_idx_j}")

        optim_vars = vertex_i, vertex_j, l_ij
        #aux_vars = relative_pose, gaussian_means_th

        cost_weight_line_process = th.ScaleCostWeight(coefficient) # for line process

        if self._requires_auto_grad:
            for point in gaussian_means:
                point_th = th.Point3(tensor=point)
                aux_vars = relative_pose, point_th
                cost_function = th.AutoDiffCostFunction(
                    optim_vars, dense_surface_alignment, 3, cost_weight_alignment, aux_vars)
                self._objective.add(cost_function)

            optim_vars = l_ij,
            cost_function = th.AutoDiffCostFunction(
                    optim_vars, line_process, 1, cost_weight_line_process) # auxiliary variables can be not declared
            self._objective.add(cost_function)
        
            self._theseus_inputs.update({
                vertex_i.name: vertex_i.tensor, 
                vertex_j.name: vertex_j.tensor,
                l_ij.name: l_ij.tensor
            })
        else:
            raise NotImplementedError()

    def optimize(self, max_iterations=1e3, step_size=0.01, track_best_solution=True, verbose=False):
        optimizer = th.LevenbergMarquardt(
            objective=self._objective,
            max_iterations=max_iterations,
            step_size=step_size)
        
        layer = th.TheseusLayer(optimizer)

        with torch.no_grad():
            _, info = layer.forward(
                self._theseus_inputs, 
                optimizer_kwargs={"track_best_solution":track_best_solution, "verbose":verbose}
                )
        return info

In [22]:
def dense_surface_alignment(
        optim_vars: Union[Tuple[th.SE3, th.SE3], Tuple[th.SE3, th.SE3, th.Vector]],
        aux_vars: Tuple[th.SE3, th.Point3]
    ) -> torch.Tensor:
    """
    Compute the dense surface alignment error between two vertices, can be used as the error
    function input to instantiate a th.CostFunction variable

    Args:
        optim_vars: optimizaiton variables registered in cost function, should contain
            pose_i, pose_j: correction matrix for pose i, j
            l_ij (optional): line process coefficient

        aux_vars: auxiliary variables registered in cost function, should contain
            relative_pose: constraint between vertex_i and vertex_j
            gaussian_means_i: mean positions of the 3D Gaussians inside camera frustum, 
                represented in coordinate i and coordinate (those in coordinate j are not needed)

    Returns:
        square root of global place recognition error
    """
    # determine whether the edge is odometry edge or loop closure edge
    tuple_size = len(optim_vars)
    if tuple_size == 2:
        pose_i, pose_j = optim_vars
    elif tuple_size == 3:
        pose_i, pose_j, l_ij = optim_vars
    else:
        raise ValueError(f"optim_vars tuple size is {tuple_size}, which can only be 2 or 3.")
    relative_pose, gaussian_means_i = aux_vars
    
    # transform all points in coordinate i and j to world coordinate
    gaussian_means_i_transformed: th.Point3 = pose_i.transform_from(gaussian_means_i)
    gaussian_means_j_transformed: th.Point3 = pose_j.transform_from(
        relative_pose.transform_from(gaussian_means_i))

    residual = (gaussian_means_i_transformed - gaussian_means_j_transformed).tensor

    # check if this error function is used for odometry edge or loop edge
    if tuple_size == 2:
        return residual
    else:
        return torch.sqrt(l_ij.tensor) * residual


def line_process(optim_vars: th.Vector, aux_vars=None) -> torch.Tensor:
    """
    Computes the line process error of a loop closrue edge, can be used as the error
    input to instantiate a th.CostFunction variable

    Args:
        optim_vars:
            l_ij: jointly optimized weight (l_ij ∈ [0, 1]) over the loop edges
            (note that the scaling factor mu is considered as cost_weight)

    Returns:
        square root of line process error
    """
    l_ij, = optim_vars
    return l_ij.tensor.sqrt() - 1


def match_gaussian_means(
        pts_1: torch.tensor,
        pts_2: torch.tensor,
        transformation: torch.tensor,
        epsilon:float=10e-2
    ) -> List[Tuple[int, int]]:
    """
    Select inlier correspondences from two Gaussian clouds, use kd-tree to speed up

    Args:
        pts_1, pts_2: mean positions of 3D Gaussians
        transformation: prior transformation matrix from one Gaussian cloud to the other
        epsilon: threshold for finding inlier correspondence

    Returns:
        a list contains tuples of matching indices
    """
    if transformation.size() != torch.Size([4, 4]):
        raise ValueError(f"The size of input transformation matrix must be (4, 4), but get {transformation.size()}")

    if pts_1.size(-1) != 1:
        pts_1 = pts_1.unsqueeze(-1)

    rotation = transformation[:3, :3]
    translation = transformation[:3, 3]
    pts_1_new = (rotation @ pts_1).squeeze() + translation

    pts_1_numpy = torch2np(pts_1_new)
    pts_2_numpy = torch2np(pts_2)
    pts2_kdtree = KDTree(pts_2_numpy)

    _, query_idx = pts2_kdtree.query(pts_1_numpy, distance_upper_bound=epsilon, workers=-1)

    data_size = pts_1.size()[0]
    res_list = []
    for i in range(data_size):
        if query_idx[i] != data_size:
            res_list.append((i, query_idx[i]))

    return res_list

In [23]:
def create_data(
        num_pts: int = 100, 
        num_poses: int = 10, 
        translation_noise: float = 0.05, 
        rotation_noise: float = 0.1, 
        batch_size: int = 1
        ) -> Tuple[List[th.Point3], List[th.SE3], List[th.SE3], List[GaussianSLAMEdge]]:
    """
    create point clouds represented in different coordinates, record their ground truth 
    absolute pose, noisy absolute pose, also return an empty list to put loop edges

    Returns:
        point_list: a list stores points clouds, represented in different coordinates
        abs_pose_list_gt: a list stores ground truth absolute poses
        abs_pose_list: a list stores noisy (odometry) absolute poses
        edge_list: a list stores custum GaussianSLAMEdge
        TODO: Do I need to put the first edge that connets vertex_0 and vertex_1 into the list?
    """

    dtype = torch.float32 # will get error if changed to torch.float64, don't know why

    points_0 = th.Point3(2*torch.rand(num_pts, 3)-1, name="POINT_CLOUD__0") # initial points in world frame
    point_list = [points_0] # represented in different frames
    abs_pose_list_gt = [] # frame i to world frame
    abs_pose_list = [] # frame i to world frame (noisy)
    edge_list = []

    abs_pose_list_gt.append(th.SE3(
        tensor=torch.tile(torch.eye(3, 4, dtype=dtype), [1, 1, 1]),
        name="VERTEX_SE3_GT__0"
        ))
    
    abs_pose_list.append(th.SE3(
        tensor=torch.tile(torch.eye(3, 4, dtype=dtype), [1, 1, 1]),
        name="VERTEX_SE3__0"
        ))

    for idx in range(1, num_poses):

        # ground truth relative pose from frame_{idx-1} to frame_{idx}
        relative_pose_gt = th.SE3.exp_map(
            torch.cat([torch.rand(batch_size, 3)-0.5, 2.0 * torch.rand(batch_size, 3)-1], dim=1),
        )

        # generate points represented in frame_{idx}
        points = relative_pose_gt.transform_from(point_list[-1])
        points.name = f"POINT_CLOUD__{idx}"
        point_list.append(points)

        # add noise to get odometry relative pose from frame_{idx-1} to frame_{idx}
        relative_pose_noise = th.SE3.exp_map(
            torch.cat([
                translation_noise * (2.0 * torch.rand(batch_size, 3) - 1),
                rotation_noise * (2.0 * torch.rand(batch_size, 3) - 1),
            ] ,dim=1),
        )

        relative_pose = cast(th.SE3, relative_pose_noise.compose(relative_pose_gt))
        relative_pose.name = f"EDGE_SE3__{idx-1}_{idx}"
        weight = th.ScaleCostWeight(1.0, name=f"EDGE_WEIGHT__{idx-1}_{idx}")

        # absolute pose of frame_{idx}
        absolute_pose_gt = cast(th.SE3, abs_pose_list_gt[-1].compose(relative_pose_gt.inverse()))
        absolute_pose_gt.name = f"VERTEX_SE3_GT__{idx}"

        absolute_pose = cast(th.SE3, abs_pose_list[-1].compose(relative_pose.inverse()))
        absolute_pose.name = f"VERTEX_SE3__{idx}"

        abs_pose_list_gt.append(absolute_pose_gt)
        abs_pose_list.append(absolute_pose)

        # construct odometry edge between vertex_{idx-1} and vertex_{idx}
        edge_list.append(GaussianSLAMEdge(idx-1, idx, relative_pose, weight))

    return point_list, abs_pose_list_gt, abs_pose_list, edge_list


def add_loop_data(
        i: int, 
        j: int, 
        abs_pose_list_gt: List[th.SE3], 
        edge_list: List[GaussianSLAMEdge],
        coefficient: float = 1.0,
        measurement_noise:float = 0.01,
        batch_size: int = 1
        ) -> None:
    """
    Add loop closure between two arbitray coordinates i and j (i < j), and stores generated edge
    """

    if i >= j:
        raise ValueError(f"The first frame index {i} is greater than the second frame index {j}!")

    abs_pose_i_gt = abs_pose_list_gt[i]
    abs_pose_j_gt = abs_pose_list_gt[j]
    rel_pose_ij_gt = th.SE3.compose(abs_pose_j_gt.inverse(), abs_pose_i_gt)
    rel_pose_ij_gt.name = f"EDGE_SE3_GT__{i}_{j}"

    relative_pose_noise = th.SE3.exp_map(
            torch.cat([
                measurement_noise * (2.0 * torch.rand(batch_size, 3) - 1),
                measurement_noise * (2.0 * torch.rand(batch_size, 3) - 1),
            ] ,dim=1),
            )
    rel_pose_ij = cast(th.SE3, relative_pose_noise.compose(rel_pose_ij_gt))
    rel_pose_ij.name = f"EDGE_SE3__{i}_{j}"

    cost_weight = th.ScaleCostWeight(coefficient, name=f"EDGE_WEIGHT__{i}_{j}")
    edge = GaussianSLAMEdge(i, j, rel_pose_ij, cost_weight)
    edge_list.append(edge)

In [27]:
point_list, abs_pose_gt_list, abs_pose_list, edge_list = create_data()
add_loop_data(1, 9, abs_pose_gt_list, edge_list)
print(f"vertex_1 before optimization: {abs_pose_list[1]}")
print(f"vertex_2 before optimization: {abs_pose_list[2]}")
print(f"vertex_3 before optimization: {abs_pose_list[3]}")
print(f"vertex_4 before optimization: {abs_pose_list[4]}")
print(f"vertex_5 before optimization: {abs_pose_list[5]}")
print(f"vertex_6 before optimization: {abs_pose_list[6]}")
print(f"vertex_7 before optimization: {abs_pose_list[7]}")
print(f"vertex_8 before optimization: {abs_pose_list[8]}")
print(f"vertex_9 before optimization: {abs_pose_list[9]}")

print("Constructing a pose graph for Gaussian Splatting SLAM.")
pose_graph = GaussianSLAMPoseGraph(requires_auto_grad=True)

for idx in range(len(edge_list)):
#for idx in range(1):
    edge = edge_list[idx]
    vertex_idx_i = edge.vertex_idx_i
    vertex_idx_j = edge.vertex_idx_j
    
    vertex_i = abs_pose_list[vertex_idx_i]
    vertex_j = abs_pose_list[vertex_idx_j]

    if vertex_idx_j - vertex_idx_i == 1:
        print(f"adding edge {idx} to pose graph, current edge is an odometry edge.")
        pose_graph.add_odometry_edge(vertex_i, vertex_j, edge, point_list[idx])
    else:
        print(f"adding edge {idx} to pose graph, current edge is an loop edge.")
        pose_i_gt, pose_j_gt = abs_pose_gt_list[vertex_idx_i], abs_pose_gt_list[vertex_idx_j]
        relative_pose_gt = th.SE3.compose(pose_j_gt.inverse(), pose_i_gt)
        inlier_idx = match_gaussian_means(point_list[vertex_idx_i].tensor, point_list[vertex_idx_j].tensor, relative_pose_gt.to_matrix().squeeze(), epsilon=5e-2)
        inlier_idx_i = [idx[0] for idx in inlier_idx]
        pose_graph.add_loop_closure_edge(vertex_i, vertex_j, edge, point_list[vertex_idx_i][inlier_idx_i], 1.0)


vertex_1: SE3(matrix=tensor([[[ 0.9150,  0.2735,  0.2967,  0.0066],
         [ 0.0284,  0.6898, -0.7234,  0.1104],
         [-0.4026,  0.6703,  0.6234, -0.5111]]])), name=VERTEX_SE3__1)
vertex_2: SE3(matrix=tensor([[[ 0.4265, -0.2118,  0.8793,  0.2306],
         [ 0.8338,  0.4689, -0.2914, -0.0354],
         [-0.3506,  0.8575,  0.3766, -0.5623]]])), name=VERTEX_SE3__2)
vertex_3: SE3(matrix=tensor([[[ 0.4105,  0.7207,  0.5587, -0.1647],
         [ 0.7539,  0.0764, -0.6525, -0.2323],
         [-0.5129,  0.6891, -0.5120, -0.3413]]])), name=VERTEX_SE3__3)
vertex_4: SE3(matrix=tensor([[[ 0.7916, -0.2911,  0.5372,  0.2179],
         [ 0.6110,  0.3723, -0.6986,  0.0327],
         [ 0.0034,  0.8813,  0.4726, -0.4036]]])), name=VERTEX_SE3__4)
vertex_5: SE3(matrix=tensor([[[ 0.8226,  0.4849,  0.2970, -0.0891],
         [ 0.3694, -0.0587, -0.9274,  0.2681],
         [-0.4322,  0.8726, -0.2274, -0.5726]]])), name=VERTEX_SE3__5)
vertex_6: SE3(matrix=tensor([[[ 0.3114, -0.4973,  0.8097,  0.2376],
  

In [28]:
#print(pose_graph._objective.error().shape)
info = pose_graph.optimize(max_iterations=1e5, step_size=0.01, verbose=False)
print(info)
print(f"vertex_1 ground truth: {abs_pose_gt_list[1]}")
print(f"vertex_2 ground truth: {abs_pose_gt_list[2]}")
print(f"vertex_3 ground truth: {abs_pose_gt_list[3]}")
print(f"vertex_4 ground truth: {abs_pose_gt_list[4]}")
print(f"vertex_5 ground truth: {abs_pose_gt_list[5]}")
print(f"vertex_6 ground truth: {abs_pose_gt_list[6]}")
print(f"vertex_7 ground truth: {abs_pose_gt_list[7]}")
print(f"vertex_8 ground truth: {abs_pose_gt_list[8]}")
print(f"vertex_9 ground truth: {abs_pose_gt_list[9]}")

NonlinearOptimizerInfo(best_solution={'VERTEX_SE3__0': tensor([[[ 0.9990,  0.0098,  0.0448, -0.0261],
         [-0.0071,  0.9982, -0.0600, -0.0721],
         [-0.0453,  0.0597,  0.9972,  0.0042]]]), 'VERTEX_SE3__1': tensor([[[ 0.8963,  0.3100,  0.3172, -0.0413],
         [ 0.0460,  0.6464, -0.7616,  0.0688],
         [-0.4412,  0.6972,  0.5651, -0.4992]]]), 'VERTEX_SE3__2': tensor([[[ 0.4231, -0.1800,  0.8880,  0.1893],
         [ 0.8464,  0.4283, -0.3165, -0.0633],
         [-0.3234,  0.8855,  0.3336, -0.5677]]]), 'VERTEX_SE3__3': tensor([[[ 0.4057,  0.7344,  0.5441, -0.1936],
         [ 0.7689,  0.0476, -0.6376, -0.2527],
         [-0.4942,  0.6770, -0.5454, -0.3435]]]), 'VERTEX_SE3__4': tensor([[[ 0.7939, -0.2835,  0.5379,  0.1972],
         [ 0.6079,  0.3535, -0.7110,  0.0272],
         [ 0.0114,  0.8915,  0.4530, -0.4021]]]), 'VERTEX_SE3__5': tensor([[[ 0.8260,  0.4797,  0.2960, -0.0998],
         [ 0.3723, -0.0699, -0.9255,  0.2799],
         [-0.4233,  0.8746, -0.2364, -0.5686]]

In [9]:
# test match_gaussian_means with hyperparameter
pose_0, pose_1 = abs_pose_list[0], abs_pose_list[1]
pose_01 = th.SE3.compose(pose_1.inverse(), pose_0).to_matrix().squeeze()
idx = match_gaussian_means(point_list[0].tensor, point_list[1].tensor, pose_01)
idx

[(0, 0),
 (1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (5, 5),
 (6, 6),
 (7, 7),
 (8, 8),
 (9, 9),
 (10, 10),
 (11, 11),
 (12, 12),
 (13, 13),
 (14, 14),
 (15, 15),
 (16, 16),
 (17, 17),
 (18, 18),
 (20, 20),
 (21, 21),
 (22, 22),
 (23, 23),
 (24, 24),
 (25, 25),
 (26, 26),
 (27, 27),
 (28, 28),
 (29, 29),
 (30, 30),
 (31, 31),
 (32, 32),
 (33, 33),
 (34, 34),
 (35, 35),
 (36, 36),
 (37, 37),
 (38, 38),
 (39, 39),
 (40, 40),
 (41, 41),
 (42, 42),
 (43, 43),
 (44, 44),
 (45, 45),
 (46, 46),
 (47, 47),
 (48, 48),
 (49, 49),
 (50, 50),
 (51, 51),
 (52, 52),
 (53, 53),
 (54, 54),
 (55, 55),
 (56, 56),
 (57, 57),
 (58, 58),
 (59, 59),
 (60, 60),
 (61, 61),
 (63, 63),
 (64, 64),
 (65, 65),
 (66, 66),
 (67, 67),
 (68, 68),
 (69, 69),
 (70, 70),
 (71, 71),
 (72, 72),
 (73, 73),
 (74, 74),
 (75, 75),
 (76, 76),
 (77, 77),
 (78, 78),
 (79, 79),
 (80, 80),
 (81, 81),
 (82, 82),
 (83, 91),
 (84, 84),
 (85, 85),
 (86, 86),
 (87, 87),
 (88, 88),
 (89, 89),
 (90, 70),
 (91, 91),
 (92, 92),
 (93, 93),
 (94, 94

In [10]:
print(point_list[0][0, :])
print(point_list[1][0, :])
print(abs_pose_gt_list[1].inverse())
print(edge_list[0].relative_pose)

tensor([ 0.7252, -0.3291,  0.4018])
tensor([1.1422, 0.6562, 0.1981])
SE3(matrix=tensor([[[ 0.8453, -0.5334, -0.0303,  0.3657],
         [ 0.3933,  0.5829,  0.7111,  0.2772],
         [-0.3616, -0.6130,  0.7025, -0.0236]]])), name=SE3__4633)
SE3(matrix=tensor([[[ 0.8714, -0.4903, -0.0183,  0.3830],
         [ 0.3392,  0.5750,  0.7445,  0.2756],
         [-0.3545, -0.6550,  0.6673, -0.0013]]])), name=EDGE_SE3__0_1)


In [11]:
pose = abs_pose_gt_list[1].inverse().tensor.squeeze()
R = pose[:3, :3]
t = pose[:3, 3]
rel_pose = edge_list[0].relative_pose.tensor.squeeze()
print(edge_list[0].vertex_idx_i)
#R = rel_pose[:3, :3]
#t = rel_pose[:3, -1]


print((R @ point_list[0][0, :].unsqueeze(-1)).squeeze()+t)

0
tensor([1.1422, 0.6562, 0.1981])
